In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stock-tweets-for-sentiment-analysis-and-prediction/stock_yfinance_data.csv
/kaggle/input/stock-tweets-for-sentiment-analysis-and-prediction/stock_tweets.csv


In [2]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
# Load data
df = pd.read_csv('/kaggle/input/stock-tweets-for-sentiment-analysis-and-prediction/stock_tweets.csv')
df.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [4]:
# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
# Perform sentiment analysis
df['scores'] = df['Tweet'].apply(lambda text: analyzer.polarity_scores(text))
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment'] = df['compound'].apply(lambda score: 'positive' if score >= 0.05 else 'negative' if score <= -0.05 else 'neutral')

In [6]:
# Specify the stock name you want to analyze
stock_name = 'TSLA'

In [7]:
# Filter the DataFrame for the specified stock
stock_data = df.loc[df['Stock Name'] == stock_name]

In [8]:
# Print sentiment scores for the specified stock
print(f"Sentiment Scores for {stock_name}:")
print(stock_data[[ 'sentiment', 'compound']])

Sentiment Scores for TSLA:
      sentiment  compound
0      positive    0.0772
1       neutral    0.0000
2      positive    0.2960
3      negative   -0.7568
4      negative   -0.8750
...         ...       ...
37417  negative   -0.1531
37418  positive    0.5719
37419   neutral    0.0000
37420  positive    0.4215
37421  positive    0.6590

[37422 rows x 2 columns]


In [9]:
# Print sentiment counts for the specified stock
print(f"Sentiment Counts for {stock_name}:")
print(stock_data['sentiment'].value_counts())

Sentiment Counts for TSLA:
sentiment
positive    18263
neutral     11132
negative     8027
Name: count, dtype: int64


In [10]:
# Aggregate sentiment scores by stock/company
sentiment_by_stock = df.groupby('Stock Name')['sentiment'].value_counts().unstack()
sentiment_by_company = df.groupby('Company Name')['sentiment'].value_counts().unstack()

In [11]:
# Print results
print("Sentiment by Stock:")
print(sentiment_by_stock)

Sentiment by Stock:
sentiment   negative  neutral  positive
Stock Name                             
AAPL            1041     1743      2272
AMD              334      641      1252
AMZN             839     1104      2146
BA                55      171       173
BX                 4       15        31
COST              66      139       188
CRM               27       93       113
DIS              112      268       255
ENPH              25       45       146
F                  9        6        16
GOOG             219      412       660
INTC              62      100       153
KO                31      124       155
META             595      877      1279
MSFT             839     1104      2146
NFLX             437      568       722
NIO              318     1211      1492
NOC                9        6        16
PG               839     1104      2146
PYPL             148      279       416
TSLA            8027    11132     18263
TSM             1972     3533      5529
VZ                14

In [12]:
print("\nSentiment by Company:")
print(sentiment_by_company)


Sentiment by Company:
sentiment                                           negative  neutral  \
Company Name                                                            
Advanced Micro Devices, Inc.                             334      641   
Alphabet Inc.                                            219      412   
Amazon.com, Inc.                                         839     1104   
Apple Inc.                                              1041     1743   
Blackstone Inc.                                            4       15   
Costco Wholesale Corporation                              66      139   
Enphase Energy, Inc.                                      25       45   
Ford Motor Company                                         9        6   
Intel Corporation                                         62      100   
Meta Platforms, Inc.                                     595      877   
Microsoft Corporation                                    839     1104   
NIO Inc.                    